# Daisy chain test

Erik Hogenbirk (ehogenbi@nikhef.nl, hogenbirk91@gmail.com)*

### Imports, settings, functions

In [2]:
# Imports
import json
import time
import os
from copy import deepcopy

In [3]:
# Configuration of the configuration maker
output_dir = '/home/xams/xams/daqcontrol/ini/'
filename = 'default_config.ini'

In [4]:
def set_register(cf, reg, val, comment="", verbose=False):
    '''This sets one register value of a config dict.'''
    # This is a list of dicts
    reg_list = cf['registers']
    new_reg_list = []
    reg_found = False
    for reg_dict in reg_list:
        # Find the dict with that register
        if reg_dict['register'] == reg:
            # If we found it, let's change it
            reg_dict['value'] = val
            if verbose: print("Just set register %s (%s) with value %s" % (reg_dict['register'],
                                                                          reg_dict['comment'], reg_dict['value']))
            # Keep track of if we found it
            reg_found = True
            new_reg_list.append(reg_dict)
        else:
            # If it doesn't match, we must just copy it
            new_reg_list.append(reg_dict)
    if not reg_found:
        # This is a NEW register, so we must build it
        print("Register %s not found in config, will append it now..." % reg)
        reg_dict = {
            "comment" : comment,
            "register": reg,
            "board" : "-1",
            "value" : val
        }
        new_reg_list.append(reg_dict)
    cf['registers'] = new_reg_list
    return cf
        

In [5]:
def save_config(cf, filename, output_dir=output_dir):
    config_path = os.path.join(output_dir, filename)
    with open(config_path, 'w') as outfile:
        json.dump(cf, outfile, indent = 4)
    print('Output written to %s' % config_path)

### Default

In [21]:
cf_default = {
    "data_folder" : "/home/xams/lena/run9/raw/data",
    "lena_pax_config_path" : "/data/xenon/xams/run9/pax_ini/_XAMSbase_light.ini",
    "zle" : False,
    "super_zle":False,
    "inverted_channels" : [],
    "links": [{
        "type": "V2718",
        "reader": 0,
        "crate": 0,
        "link": 0
    },
    {
        "type": "V2718",
        "reader": 0,
        "crate": 1,
        "link": 0
    },
    ],

    "boards": [
        {
        "crate": 0,
        "serial": "1730",
        "reader": 0,
        "type": "V1724",
        "vme_address": "Whatever?", # 3 2 1 0 for switches 2,4,5,6 on the V1730D # 2 2 2 4 for the V1724
        "link": 0,
#         "id" : 0,
#         "node" : 0
        },
        {
        "crate": 1,
        "serial": "1724",
        "reader": 0,
        "type": "V1724",
        "vme_address": "Whatever?", # 3 2 1 0 for switches 2,4,5,6 on the V1730D # 2 2 2 4 for the V1724
        "link": 0,
#         "id" : 1,
#         "node" : 1
        },
    
    ],
    # blt_size {int} : block transfer size in bytes (decimal).
    # V1724 standard: 524288
    # V1730 maximum: 2097152?
    "blt_size": 524288,
    # run_start {int} : 0 - board internal (run start via register), 
    # 1 - s-in (for synchronized boards)
    "run_start": 0,
    "baseline_mode": 0,
    # write_mode {int}: 0-no writing, 1-to file, 2-mongodb
    "write_mode": 2,

    "mongo": {
        "write_concern": 0,
        "min_insert_size": 1,
        "database": "xamsdata0",
        "address": "mongodb://localhost:27017",
        "collection": "onzin"
    },
    "processing_readout_threshold": 0,
    "processing_num_threads": 8,
    "processing_mode": 3,
    "occurrence_integral": 0,

    "compression": 1,
    
    "registers": [
        {
            "comment": "Reset software (all registers)",
            "register": "EF24",
            "board": "-1",
            "value": "1"
        },
        {
            "comment": "BLT event number",
            "register": "EF1C",
            "board": "-1",
            "value": "1"
            # Maximum number of events loaded into each block.
            # 40hex = 64dec
        },
        {
            "comment": "Enable BERR (error messages)",
            "register": "EF00",
            "board": "-1",
            "value": "10"
        },
        {
            "comment": "Aquisition control register",
            "register": "8100",
            "board": "-1",
            "value": "5" 
            #5: s-in control, 0: board control
        },
        {
            "comment": "Channel calibration",
            "register": "809C",
            "board": "-1",
            "value": "1"
            # enables channel calibration
        },
        {
            "comment": "Channel enable mask",
            "register": "8120",
            "board": "-1",
            "value": "01"
            # set to 01 to enable channel 0 only
            # set to 09 to enable channel 0 and 3
        },
        {
            "comment": "Measurement time after trigger",
            "register": "8114",
            "board": "-1",
            "value": "80"
            # 20hex = 32dec = 320 samples = 640 ns
            # 1000 = 81 920 ns
        },
        {
            "comment": "Trigger source enable mask",
            "register": "810C",
            "board": "1730",
            "value": "C0F00000"
        },
        {
            "comment": "Trigger source enable mask",
            "register": "810C",
            "board": "1724",
            "value": "80F00001"
        },
        {
            "comment": "Buffer organisation",
            "register": "800C",
            "board": "-1",
            "value": "7"
        },
        {
            "comment": "Interrupt status ID",
            "register": "EF14",
            "board": "-1",
            "value": "55AA"
            # Interrupt
        },
        {
            "comment": "Interrupt event number",
            "register": "EF18",
            "board": "-1",
            "value": "1"
            # Interrupt when 1 event stored
        },
# #         {
# #             "comment": "Channel 0 gain",
# #             "register": "1028",
# #             "board": "-1",
# #             "value": "0"
# #             # Channel n gain 0x1n28 (either 0 (2 Vpp) or 1 (0.5 Vpp))
# #             # ONLY V1730
# #         },
# #         {
# #             "comment": "Channel 3 gain",
# #             "register": "1328",
# #             "board": "-1",
# #             "value": "0"
# #             # Channel n gain 0x1n28 (either 0 (2 Vpp) or 1 (0.5 Vpp))
# #             # ONLY V1730
# #         },
        {
            "comment": "Trigger level channel 0",
            "register": "1080",
            "board": "1724",
            "value": "20BC"
            # 1FF4 (hex) = 8180 (dec)
            # 1FEA (hex) = 8170 (dec)
            # 1FE0 (hex) = 8160 (dec)
            # 1FD6 (hex) = 8150 (dec)
            # 1F40 (hex) = 8000 (dec)
            # 1E78 (hex) = 7800 (dec)
        },
        {
            "comment": "Trigger level channel 3",
            "register": "1380",
            "board": "-1",
            "value": "0000"
        },
        {
            "comment": "Trigger polarity",
            "register": "8000",
            "board": "1724",
            "value": "20010"
            # Set to 50 for negative, 10 for positive
            # 20010 for ZLE enabled
        },
        {
            "comment": "Channel 0 ZLE threshold",
            "register": "1024",
            "board": "1724",
            "value": "20B2"
        },
        {
            "comment": "Channel 0 ZLE extension",
            "register": "1028",
            "board": "1724",
            "value": "140014"
        }
        
    ],
}


save_config(cf_default, 'daisytest_V1724.ini')


Output written to /home/xams/xams/daqcontrol/ini/daisytest_V1724.ini


In [ ]:
# V1724

In [ ]:
"registers": [
        {
            "comment": "Reset software (all registers)",
            "register": "EF24", # Same for V1724
            "board": "-1",
            "value": "1"
        },
        {
            "comment": "BLT event number",
            "register": "EF1C", # Same for V1724
            "board": "-1",
            "value": "1"
            # Maximum number of events loaded into each block.
            # 40hex = 64dec
        },
        {
            "comment": "Enable BERR (error messages)",
            "register": "EF00", # Same for V1724
            "board": "-1",
            "value": "10"
        },
        {
            "comment": "Aquisition control register",
            "register": "8100", # Same for V1724
            "board": "-1",
            "value": "5" 
            #5: s-in control, 0: board control
        },
        {
            "comment": "Channel calibration",
            "register": "809C", # Not found in documentation
            "board": "-1",
            "value": "1"
            # enables channel calibration
        },
        {
            "comment": "Channel enable mask",
            "register": "8120", # Same in V1724
            "board": "-1",
            "value": "09"
            # set to 01 to enable channel 0 only
            # set to 09 to enable channel 0 and 3
        },
        {
            "comment": "Measurement time after trigger",
            "register": "8114", # Same in V1724
            "board": "-1",
            "value": "1400"
            # 20hex = 32dec = 320 samples = 640 ns
            # 1000 = 81 920 ns
        },
        {
            "comment": "Trigger source enable mask",
            "register": "810C",  # Same in V1724
            "board": "-1",
            "value": "C1F00003"
            #"value": "C1F00003" 120 ns
            # First digit: C for enable external + software trig
            # Second: coincidence level
            # Last one (?): channels enable
            # Set to "C0000001" for only trigger 0 and 1
        },
        {
            "comment": "Buffer organisation",
            "register": "800C", # Same in V1724
            "board": "-1",
            "value": "6"
        },
        {
            "comment": "Interrupt status ID",
            "register": "EF14", # Same in V1724
            "board": "-1",
            "value": "55AA"
            # Interrupt
        },
        {
            "comment": "Interrupt event number",
            "register": "EF18", # Same in V1724
            "board": "-1",
            "value": "1"
            # Interrupt when 1 event stored
        },
        {
            "comment": "Channel 0 gain",
            "register": "1028",
            "board": "-1",
            "value": "0"
            # Channel n gain 0x1n28 (either 0 (2 Vpp) or 1 (0.5 Vpp))
            # ONLY V1730
        },
        {
            "comment": "Channel 3 gain",
            "register": "1328",
            "board": "-1",
            "value": "0"
            # Channel n gain 0x1n28 (either 0 (2 Vpp) or 1 (0.5 Vpp))
            # ONLY V1730
        },
        {
            "comment": "Trigger level channel 0",
            "register": "1080",
            "board": "-1",
            "value": "1FBE"
            # 1FF4 (hex) = 8180 (dec)
            # 1FEA (hex) = 8170 (dec)
            # 1FE0 (hex) = 8160 (dec)
            # 1FD6 (hex) = 8150 (dec)
            # 1F40 (hex) = 8000 (dec)
            # 1E78 (hex) = 7800 (dec)
        },
        {
            "comment": "Trigger level channel 3",
            "register": "1380",
            "board": "-1",
            "value": "2004"
        },
        {
            "comment": "Trigger polarity",
            "register": "8000",
            "board": "-1",
            "value": "40"
            # Set to 40 for negative, 00 for positive
        }